In [2]:
import pandas as pd
import numpy as np
import glob as glob
import os
import time
import pathlib

# from ..route_imports import ratio_map
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
# import route_imports.ratio_map as ratios # store_load is a file on my library folder
from route_imports.ratio_map import metric_to_url_map # as ratios
from route_imports.ratio_map import url_to_metric_map
from route_imports.ratio_map import url_to_name_map
from route_imports.ratio_map import fin_statement_raw_names
from route_imports.ratio_map import fin_statement_renamed_cols
from route_imports.ratio_map import metric_to_formula_map

# from route_imports import ratio_map.metric_to_url_map# as ratios

start_time = time.time()
uri_list = "cash-and-cash-equivalents"
url_symbol_name = "apple"
url_symbol = "JAX"
url_fin_metric = "cash-and-cash-equivalents"

def magnitude_num(number, currency_symbol):
    if len(str(number)) > 9:
        magnitude = number/1000000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"B")
    elif len(str(number)) > 6:
        magnitude = number/1000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"M")
    elif len(str(number)) > 3:
        magnitude = number/1000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    else:
        magnitude = number
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    return magnitude_str

In [3]:
company_profiles = pd.read_csv("../reference_data/Company_Profiles.csv")#, encoding='cp1252')
fin_statements_matching = pd.read_csv("../reference_data/Financial_Statements_Reference_Matching.csv")#, encoding='cp1252')
currency_symbol = list(company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())]['currency symbol'])[0]
company_profiles_col = ['symbol',
                        'long name',
                        'currency',
                        'exchange',
                        'industry',
                        'description',
                        'sector',
                        'country',
                        'ipo date',
                        'short name']
company_profiles = company_profiles[company_profiles_col]  

In [23]:
# hide - cash equiv
profiles_dict = {}
profiles_value = company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())].values.tolist()[0]

titles_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['Title'])
titles_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['Title'])
titles_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['Title'])

urls_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['URL'])
urls_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['URL'])
urls_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['URL'])

In [24]:
# hide - cash equiv
for n, profiles_col in enumerate(company_profiles_col):
    key = profiles_col
    value = profiles_value[n]
    profiles_dict[key] = value 

if "{}".format(url_fin_metric) in urls_is:
    fin_statement_dir = "Income Statement"
elif "{}".format(url_fin_metric) in urls_bs:
    fin_metric_pos = urls_bs.index("{}".format(url_fin_metric))
    fin_statement_dir = "Balance Sheet"
    fin_statement_cols = titles_bs
    cols = titles_bs
    fin_metric_title = fin_statement_cols[fin_metric_pos]

elif "{}".format(url_fin_metric) in urls_cf:
    fin_statement_dir = "Cash Flow Statement"

else:
    pass


In [25]:
# hide - cash equiv
# start_time = time.time()
bs_dir = "Balance Sheet"
bs_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/quarter/{}/*~{}~*".format(bs_dir, url_symbol.upper()))[-1]

df = pd.read_csv(bs_file,index_col = False) #.format("NLOK"))[-1]
# df = df[df['date'].notna()]#fillna(method='ffill')
df = df[0:].iloc[::-1]#.dropna()
df_fin_statement = df
#region Pandas data manipulation
matching_row = fin_statements_matching[fin_statements_matching['URL']=="{}".format(url_fin_metric)]
fin_metric_title = list(matching_row['Title'])[0]
fin_metric_name = list(matching_row['Name'])[0]
print("fin metric name", fin_metric_name)
sorted_metric = df["{}".format(fin_metric_name)].sort_values()

fin metric name cashAndCashEquivalents


In [10]:
fin_dir = ["Income Statement","Balance Sheet","Cash Flow Statement"]
# url_symbol = "TSLA"
url_symbol = "JAX"

fin_df_list = []
for x in fin_dir:
    fin_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/quarter/{}/*~{}~*".format(x, url_symbol.upper()))[-1]
    fin_df = pd.read_csv(fin_file)
    fin_df["Year"] = fin_df["date"].apply(lambda x: x[0:4])
    fin_df["QQ-YYYY"] = fin_df["period"]+"-"+fin_df["date"].apply(lambda x: x[0:4])#["QQ-YYYY"]
    fin_df_list.append(fin_df)

In [11]:
fin_df_list[1]["period"]+"-"+fin_df_list[1]["date"].apply(lambda x: x[0:4])#["QQ-YYYY"]

0     Q3-2020
1     Q2-2020
2     Q1-2020
3     Q4-2019
4     Q3-2019
5     Q2-2019
6     Q1-2019
7     Q4-2018
8     Q3-2018
9     Q2-2018
10    Q1-2018
11    Q4-2017
12    Q3-2017
13    Q2-2017
14    Q1-2017
15    Q4-2017
16    Q3-2016
17    Q2-2016
18    Q1-2016
19    Q4-2016
20    Q3-2015
21    Q2-2015
dtype: object

In [12]:
# df = pd.concat(fin_df_list,ignore_index=True)#,axis = 1)
# df = pd.concat(fin_df_list, join='outer', axis=1)#.fillna(nan_value)

# df = pd.concat(fin_df_list,ignore_index=True)
df_merge_is_bs = pd.merge(fin_df_list[0],fin_df_list[1],how="inner", on="QQ-YYYY")
# df
# df_merge cols
df_merge = pd.merge(df_merge_is_bs,fin_df_list[2],how="inner", on="QQ-YYYY")
df_merge = df_merge[fin_statement_raw_names]

df_merge.columns = fin_statement_renamed_cols
df = df_merge

In [14]:
df['ffo_math']=df['net_income'] + df['d_n_a'] + df['sales_maturities_of_investments'] + df['purchase_of_investments'] + df['investments_in_pp_n_e'] + df['acquisitions_net']
df['book_value_math']=df['total_assets'].dropna()-df['total_liabilities'].dropna()
df['ebit_math']=df['ebitda_non'] - df['d_n_a']
df['working_capital_math']= df['total_current_assets'] - df['total_current_liabilities']

df['quick_assets_math']=df['cash_non']+df['short_term_investments']+df['accounts_receivable']
df['quick_ratio_math']=df['total_current_assets'] - df['inventory']
# df['other_current_assets'] * df['total_current_assets']
# df['g_n_a']+df['s_n_m']

In [25]:
# wtf red dot
# perc​entage_of_debt_to_asset_formula = lambda perc​entage_of_debt_to_asset_formula: ((total_non_current_liabilities)/(total_assets))
# rece​ivables_turnover = lambda rece​ivables_turnover: ((net_revenue)/(accounts_receivable))

In [26]:
# # hide lambda
# net_working_capital_ratio = lambda net_working_capital_ratio: (df['working_capital_math']/df['total_assets'])
# book_value_per_share = lambda book_value_per_share: (df['total_se']/df['shares_outstanding_non'])
# total_equity_to_total_assets = lambda total_equity_to_total_assets: (df['total_se']/df['total_assets'])
# operating_cost_ratio = lambda operating_cost_ratio: (df['total_opex'] - df['d_n_a']/df['net_revenue'])
# percentage_of_debt_to_asset_formula = lambda percentage_of_debt_to_asset_formula: (df['total_non_current_liabilities']/df['total_assets'])

# total_liabilities_pct_of_total_assets = lambda total_liabilities_pct_of_total_assets: (df['total_liabilities']/df['total_assets'])
# debt_to_assets_ratio = lambda debt_to_assets_ratio: (df['total_debt']/df['total_assets'])
# debt_to_equity_ratio = lambda debt_to_equity_ratio: (df['total_debt']/df['total_se'])
# quick_ratio = lambda quick_ratio: (df['total_current_assets']-df['inventory']/df['total_current_liabilities'])
# working_capital = lambda working_capital: (df['total_current_assets'] - df['total_current_liabilities'])
# current_ratio = lambda current_ratio: (df['total_current_assets']/df['total_current_liabilities'])
# capital_intensity = lambda capital_intensity: (df['total_assets']/df['net_revenue'])
# equity_multiplier = lambda equity_multiplier: (df['total_assets']/df['total_se'])
# short_term_debt_to_equity_ratio = lambda short_term_debt_to_equity_ratio: (df['short_term_debt']/df['total_se'])
# st_debt_as_pct_of_total_debt = lambda st_debt_as_pct_of_total_debt: (df['short_term_debt']/df['total_liabilities'])
# acid_test_ratio = lambda acid_test_ratio: (df['quick_assets_math']/df['total_current_liabilities'])
# pre_tax_income_to_sales = lambda pre_tax_income_to_sales: (df['pretax_income_non']/df['net_revenue'])
# pre_tax_return_on_assets = lambda pre_tax_return_on_assets: (df['pretax_income_non']/df['total_assets'])
# pre_tax_return_on_common_equity = lambda pre_tax_return_on_common_equity: (df['pretax_income_non']/df['common_stock'])
# operating_roa = lambda operating_roa: (df['operating_income']/df['total_assets'])
# operating_profit_margin = lambda operating_profit_margin: (df['operating_income']/df['net_revenue'])
# free_operating_cash_flow_to_debt = lambda free_operating_cash_flow_to_debt: (df['operating_cash_flow']-df['capex']/df['total_debt'])
# discretionary_cash_flow_to_debt = lambda discretionary_cash_flow_to_debt: (df['operating_cash_flow']-df['accounts_payable']-df['debt_repayment']-df['dividends_paid']-df['interest_expense'])
# operating_cash_flow_to_interest = lambda operating_cash_flow_to_interest: (df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['interest_expense'])
# operating_cash_flow_to_debt = lambda operating_cash_flow_to_debt: (df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['total_liabilities'])
# cash_flow_margin_ratio_formula = lambda cash_flow_margin_ratio_formula: (df['operating_cash_flow']/df['total_liabilities'])
# cash_flow_to_debt = lambda cash_flow_to_debt: (df['operating_cash_flow']/df['total_debt'])
# net_cash_flow_to_capital_expenditures = lambda net_cash_flow_to_capital_expenditures: (df['operating_cash_flow']/df['capex'])
# cash_flow_to_revenue = lambda cash_flow_to_revenue: (df['operating_cash_flow']/df['net_revenue'])
# cash_return_on_assets = lambda cash_return_on_assets: (df['operating_cash_flow']/df['total_assets'])
# cash_return_on_equity = lambda cash_return_on_equity: (df['operating_cash_flow']/df['total_se'])
# cash_to_income_ratio = lambda cash_to_income_ratio: (df['operating_cash_flow']/df['operating_income'])
# cash_flow_per_share = lambda cash_flow_per_share: (df['operating_cash_flow']/df['shares_outstanding_non'])
# debt_payment = lambda debt_payment: (df['operating_cash_flow']/df['debt_repayment'])
# debt_coverage = lambda debt_coverage: (df['operating_cash_flow']/df['long_term_debt'])
# cash_flow_from_operations_ratio = lambda cash_flow_from_operations_ratio: (df['operating_cash_flow']/df['total_current_liabilities'])
# gross_profit_margin = lambda gross_profit_margin: (df['net_revenue'] - df['cost_of_sales']/df['net_revenue'])
# receivables_turnover = lambda receivables_turnover: (df['net_revenue']/df['accounts_receivable'])
# capital_turnover_ratio = lambda capital_turnover_ratio: (df['net_revenue']/df['working_capital_math'])
# assets_turnover_ratio = lambda assets_turnover_ratio: (df['net_revenue']/df['total_assets'])
# accounts_receivableturnover = lambda accounts_receivableturnover: (df['net_revenue']/df['accounts_receivable'])
# operating_cash_flow_to_debt = lambda operating_cash_flow_to_debt: (df['net_revenue']/df['total_se'])
# inventory_ratio = lambda inventory_ratio: (df['net_revenue']/df['inventory'])
# return_on_investment = lambda return_on_investment: (df['net_income']+ df['interest_expense']/df['total_se'])
# pretax_margin = lambda pretax_margin: (df['net_income'] - df['income_tax_expense']/df['net_revenue'])
# income_to_net_worth_ratio = lambda income_to_net_worth_ratio: (df['net_income'] - df['deferred_income_tax']/df['shares_outstanding_non'])
# return_on_assets_roa = lambda return_on_assets_roa: (df['net_income']/df['total_assets'])
# roe = lambda roe: (df['net_income']/df['total_se'])
# profit_margin = lambda profit_margin: (df['net_income']/df['net_revenue'])
# earnings_per_share = lambda earnings_per_share: (df['net_income']/df['shares_outstanding_non'])
# current_cash_debt_coverage = lambda current_cash_debt_coverage: (df['net_cash_by_operating_activities']/df['total_current_liabilities'])
# cash_debt_coverage = lambda cash_debt_coverage: (df['net_cash_by_operating_activities']/df['total_liabilities'])
# long_term_debt_ratio = lambda long_term_debt_ratio: (df['long_term_debt']/df['total_assets'])
# long_term_debt_equity_ratio = lambda long_term_debt_equity_ratio: (df['long_term_debt']/df['total_se'])
# lt_debt_as_pct_of_total_debt = lambda lt_debt_as_pct_of_total_debt: (df['long_term_debt']/df['total_liabilities'])
# inventory_pct_of_revenue = lambda inventory_pct_of_revenue: (df['inventory']/df['net_revenue'])
# intangibles_pct_of_book_value = lambda intangibles_pct_of_book_value: (df['goodwill_n_intangible_assets']/df['total_se'])
# ffo_funds_from_operations_to_debt = lambda ffo_funds_from_operations_to_debt: (df['ffo_math']/df['total_debt'])
# operating_margin = lambda operating_margin: (df['ebitda_non']-df['d_a']/df['net_revenue'])
# cash_coverage_ratio = lambda cash_coverage_ratio: (df['ebitda_non']/df['interest_expense'])
# ebitda_per_share = lambda ebitda_per_share: (df['ebitda_non']/df['shares_outstanding_non'])
# ebitda_interest_coverage = lambda ebitda_interest_coverage: (df['ebitda_non']/df['interest_expense'])
# ebitda_margin = lambda ebitda_margin: (df['ebitda_non']/df['net_revenue'])
# net_margin_profit_margin = lambda net_margin_profit_margin: (df['ebitda_margin']/df['net_revenue'])
# return_on_capital_employed_ratio = lambda return_on_capital_employed_ratio: (df['ebit_math']/df['total_assets'] - df['total_current_liabilities'])
# debt_service_ratio = lambda debt_service_ratio: (df['ebit_math']/df['interest_expense'])
# return_on_capital = lambda return_on_capital: (df['ebit_math']/df['total_assets'])
# dividend_yield = lambda dividend_yield: (df['dividends_paid']/df['shares_outstanding_non'])
# dividend_payout_ratio = lambda dividend_payout_ratio: (df['dividends_paid']/df['net_income'])
# dividends_per_share = lambda dividends_per_share: (df['dividends_paid']/df['shares_outstanding_non'])
# inventory_turnover = lambda inventory_turnover: (df['cost_of_sales']/df['inventory'])
# cash_ratio = lambda cash_ratio: (df['cash_non']/df['total_current_liabilities'])
# cash_flow_roa = lambda cash_flow_roa: (df['operating_cash_flow']/df['total_assets'])
# average_collection_period = lambda average_collection_period: (df['accounts_receivable']/df['net_revenue'])
# number_of_days_of_receivables = lambda number_of_days_of_receivables: (df['accounts_receivable']/df['inventory'])
# average_days_payables_outstanding = lambda average_days_payables_outstanding: (df['accounts_payable']/df['cost_of_sales'])
# days_sales_in_payables = lambda days_sales_in_payables: (df['accounts_payable']/df['total_opex'])

In [27]:
# ratios_df = pd.read_csv("../reference_data/ratios.csv")
# ratios_df = ratios_df.dropna(subset = ["Name"])
# fin_statements_matching[fin_statements_matching['URL']=="{}".format(url_fin_metric)]
# fin_metric_ratio_variable_name = list(matching_row['Python Variable Name'])[0]
# # ratios_df = pd.read_csv("../reference_data/ratios.csv")



In [28]:
# # metric to url mapping
# metric_to_url_map = {
#     'net_working_capital_ratio': "net-working-capital-ratio",
#     'book_value_per_share': "book-value-per-share",
#     'total_equity_to_total_assets': "total-equity-to-total-assets",
#     'operating_cost_ratio': "operating-cost-ratio",
#     'percentage_of_debt_to_asset_formula': "perc​entage-of-debt-to-asset-formula",

#     'total_liabilities_pct_of_total_assets': "total-liabilities-pct-of-total-assets",
#     'debt_to_assets_ratio': "debt-to-assets-ratio",
#     'debt_to_equity_ratio': "debt-to-equity-ratio",
#     'quick_ratio': "quick-ratio",
#     'working_capital': "working-capital",
#     'current_ratio': "current-ratio",
#     'capital_intensity': "capital-intensity",
#     'equity_multiplier': "equity-multiplier",
#     'short_term_debt_to_equity_ratio': "short-term-debt-to-equity-ratio",
#     'st_debt_as_pct_of_total_debt': "st-debt-as-pct-of-total-debt",
#     'acid_test_ratio': "acid-test-ratio",
#     'pre_tax_income_to_sales': "pre-tax-income-to-sales",
#     'pre_tax_return_on_assets': "pre-tax-return-on-assets",
#     'pre_tax_return_on_common_equity': "pre-tax-return-on-common-equity",
#     'operating_roa': "operating-roa",
#     'operating_profit_margin': "operating-profit-margin",
#     'free_operating_cash_flow_to_debt': "free-operating-cash-flow-to-debt",
#     'discretionary_cash_flow_to_debt': "discretionary-cash-flow-to-debt",
#     'operating_cash_flow_to_interest': "operating-cash-flow-to-interest",
#     'operating_cash_flow_to_debt': "operating-cash-flow-to-debt",
#     'cash_flow_margin_ratio_formula': "cash-flow-margin-ratio-formula",
#     'cash_flow_to_debt': "cash-flow-to-debt",
#     'net_cash_flow_to_capital_expenditures': "net-cash-flow-to-capital-expenditures",
#     'cash_flow_to_revenue': "cash-flow-to-revenue",
#     'cash_return_on_assets': "cash-return-on-assets",
#     'cash_return_on_equity': "cash-return-on-equity",
#     'cash_to_income_ratio': "cash-to-income-ratio",
#     'cash_flow_per_share': "cash-flow-per-share",
#     'debt_payment': "debt-payment",
#     'debt_coverage': "debt-coverage",
#     'cash_flow_from_operations_ratio': "cash-flow-from-operations-ratio",
#     'gross_profit_margin': "gross-profit-margin",
#     'receivables_turnover': "rece​ivables-turnover",
#     'capital_turnover_ratio': "capital-turnover-ratio",
#     'assets_turnover_ratio': "assets-turnover-ratio",
#     'accounts_receivableturnover': "accounts-receivableturnover",
#     'operating_cash_flow_to_debt': "operating-cash-flow-to-debt",
#     'inventory_ratio': "inventory-ratio",
#     'return_on_investment': "return-on-investment",
#     'pretax_margin': "pretax-margin",
#     'income_to_net_worth_ratio': "income-to-net-worth-ratio",
#     'return_on_assets_roa': "return-on-assets-roa",
#     'roe': "roe",
#     'profit_margin': "profit-margin",
#     'earnings_per_share': "earnings-per-share",
#     'current_cash_debt_coverage': "current-cash-debt-coverage",
#     'cash_debt_coverage': "cash-debt-coverage",
#     'long_term_debt_ratio': "long-term-debt-ratio",
#     'long_term_debt_equity_ratio': "long-term-debt-equity-ratio",
#     'lt_debt_as_pct_of_total_debt': "lt-debt-as-pct-of-total-debt",
#     'inventory_pct_of_revenue': "inventory-pct-of-revenue",
#     'intangibles_pct_of_book_value': "intangibles-pct-of-book-value",
#     'ffo_funds_from_operations_to_debt': "ffo-funds-from-operations-to-debt",
#     'operating_margin': "operating-margin",
#     'cash_coverage_ratio': "cash-coverage-ratio",
#     'ebitda_per_share': "ebitda-per-share",
#     'ebitda_interest_coverage': "ebitda-interest-coverage",
#     'ebitda_margin': "ebitda-margin",
#     'net_margin_profit_margin': "net-margin-profit-margin",
#     'return_on_capital_employed_ratio': "return-on-capital-employed-ratio",
#     'debt_service_ratio': "debt-service-ratio",
#     'return_on_capital': "return-on-capital",
#     'dividend_yield': "dividend-yield",
#     'dividend_payout_ratio': "dividend-payout-ratio",
#     'dividends_per_share': "dividends-per-share",
#     'inventory_turnover': "inventory-turnover",
#     'cash_ratio': "cash-ratio",
#     'cash_flow_roa': "cash-flow-roa",
#     'average_collection_period': "average-collection-period",
#     'number_of_days_of_receivables': "number-of-days-of-receivables",
#     'average_days_payables_outstanding': "average-days-payables-outstanding",
#     'days_sales_in_payables': "days-sales-in-payables",
#     }

In [29]:
# url_to_metric_map = {
#     'net-working-capital-ratio': "net_working_capital_ratio",
#     'book-value-per-share': "book_value_per_share",
#     'total-equity-to-total-assets': "total_equity_to_total_assets",
#     'operating-cost-ratio': "operating_cost_ratio",
#     'perc​entage-of-debt-to-asset-formula': "perc​entage_of_debt_to_asset_formula",

#     'total-liabilities-pct-of-total-assets': "total_liabilities_pct_of_total_assets",
#     'debt-to-assets-ratio': "debt_to_assets_ratio",
#     'debt-to-equity-ratio': "debt_to_equity_ratio",
#     'quick-ratio': "quick_ratio",
#     'working-capital': "working_capital",
#     'current-ratio': "current_ratio",
#     'capital-intensity': "capital_intensity",
#     'equity-multiplier': "equity_multiplier",
#     'short-term-debt-to-equity-ratio': "short_term_debt_to_equity_ratio",
#     'st-debt-as-pct-of-total-debt': "st_debt_as_pct_of_total_debt",
#     'acid-test-ratio': "acid_test_ratio",
#     'pre-tax-income-to-sales': "pre_tax_income_to_sales",
#     'pre-tax-return-on-assets': "pre_tax_return_on_assets",
#     'pre-tax-return-on-common-equity': "pre_tax_return_on_common_equity",
#     'operating-roa': "operating_roa",
#     'operating-profit-margin': "operating_profit_margin",
#     'free-operating-cash-flow-to-debt': "free_operating_cash_flow_to_debt",
#     'discretionary-cash-flow-to-debt': "discretionary_cash_flow_to_debt",
#     'operating-cash-flow-to-interest': "operating_cash_flow_to_interest",
#     'operating-cash-flow-to-debt': "operating_cash_flow_to_debt",
#     'cash-flow-margin-ratio-formula': "cash_flow_margin_ratio_formula",
#     'cash-flow-to-debt': "cash_flow_to_debt",
#     'net-cash-flow-to-capital-expenditures': "net_cash_flow_to_capital_expenditures",
#     'cash-flow-to-revenue': "cash_flow_to_revenue",
#     'cash-return-on-assets': "cash_return_on_assets",
#     'cash-return-on-equity': "cash_return_on_equity",
#     'cash-to-income-ratio': "cash_to_income_ratio",
#     'cash-flow-per-share': "cash_flow_per_share",
#     'debt-payment': "debt_payment",
#     'debt-coverage': "debt_coverage",
#     'cash-flow-from-operations-ratio': "cash_flow_from_operations_ratio",
#     'gross-profit-margin': "gross_profit_margin",
#     'rece​ivables-turnover': "rece​ivables_turnover",
#     'capital-turnover-ratio': "capital_turnover_ratio",
#     'assets-turnover-ratio': "assets_turnover_ratio",
#     'accounts-receivableturnover': "accounts_receivableturnover",
#     'operating-cash-flow-to-debt': "operating_cash_flow_to_debt",
#     'inventory-ratio': "inventory_ratio",
#     'return-on-investment': "return_on_investment",

#     'pretax-margin': "pretax_margin",
#     'income-to-net-worth-ratio': "income_to_net_worth_ratio",
#     'return-on-assets-roa': "return_on_assets_roa",
#     'roe': "roe",
#     'profit-margin': "profit_margin",
#     'earnings-per-share': "earnings_per_share",
#     'current-cash-debt-coverage': "current_cash_debt_coverage",
#     'cash-debt-coverage': "cash_debt_coverage",
#     'long-term-debt-ratio': "long_term_debt_ratio",
#     'long-term-debt-equity-ratio': "long_term_debt_equity_ratio",
#     'lt-debt-as-pct-of-total-debt': "lt_debt_as_pct_of_total_debt",
#     'inventory-pct-of-revenue': "inventory_pct_of_revenue",
#     'intangibles-pct-of-book-value': "intangibles_pct_of_book_value",
#     'ffo-funds-from-operations-to-debt': "ffo_funds_from_operations_to_debt",
#     'operating-margin': "operating_margin",
#     'cash-coverage-ratio': "cash_coverage_ratio",
#     'ebitda-per-share': "ebitda_per_share",
#     'ebitda-interest-coverage': "ebitda_interest_coverage",
#     'ebitda-margin': "ebitda_margin",
#     'net-margin-profit-margin': "net_margin_profit_margin",
#     'return-on-capital-employed-ratio': "return_on_capital_employed_ratio",
#     'debt-service-ratio': "debt_service_ratio",
#     'return-on-capital': "return_on_capital",
#     'dividend-yield': "dividend_yield",
#     'dividend-payout-ratio': "dividend_payout_ratio",
#     'dividends-per-share': "dividends_per_share",
#     'inventory-turnover': "inventory_turnover",
#     'cash-ratio': "cash_ratio",
#     'cash-flow-roa': "cash_flow_roa",
#     'average-collection-period': "average_collection_period",
#     'number-of-days-of-receivables': "number_of_days_of_receivables",
#     'average-days-payables-outstanding': "average_days_payables_outstanding",
#     'days-sales-in-payables': "days_sales_in_payables",
#     }

In [30]:
# metric to formula mapping
# fin_statements_matching
# company_profiles

# https://stackoverflow.com/questions/16756174/python-one-line-function-definition
# metric_to_formula_map = {
#     'net_working_capital_ratio':df['working_capital_math']/df['total_assets'],
#     'book_value_per_share':df['total_se']/df['shares_outstanding_non'],
#     'total_equity_to_total_assets':df['total_se']/df['total_assets'],
#     'operating_cost_ratio':df['total_opex'] - df['d_n_a']/df['net_revenue'],
#     'perc​entage_of_debt_to_asset_formula':df['total_non_current_liabilities']/df['total_assets'],

#     'total_liabilities_pct_of_total_assets':df['total_liabilities']/df['total_assets'],
#     'debt_to_assets_ratio':df['total_debt']/df['total_assets'],
#     'debt_to_equity_ratio':df['total_debt']/df['total_se'],
#     'quick_ratio':df['total_current_assets']-df['inventory']/df['total_current_liabilities'],
#     'working_capital':df['total_current_assets'] - df['total_current_liabilities'],
#     'current_ratio':df['total_current_assets']/df['total_current_liabilities'],
#     'capital_intensity':df['total_assets']/df['net_revenue'],
#     'equity_multiplier':df['total_assets']/df['total_se'],
#     'short_term_debt_to_equity_ratio':df['short_term_debt']/df['total_se'],
#     'st_debt_as_pct_of_total_debt':df['short_term_debt']/df['total_liabilities'],
#     'acid_test_ratio':df['quick_assets_math']/df['total_current_liabilities'],
#     'pre_tax_income_to_sales':df['pretax_income_non']/df['net_revenue'],
#     'pre_tax_return_on_assets':df['pretax_income_non']/df['total_assets'],
#     'pre_tax_return_on_common_equity':df['pretax_income_non']/df['common_stock'],
#     'operating_roa':df['operating_income']/df['total_assets'],
#     'operating_profit_margin':df['operating_income']/df['net_revenue'],
#     'free_operating_cash_flow_to_debt':df['operating_cash_flow']-df['capex']/df['total_debt'],
#     'discretionary_cash_flow_to_debt':df['operating_cash_flow']-df['accounts_payable']-df['debt_repayment']-df['dividends_paid']-df['interest_expense'],
#     'operating_cash_flow_to_interest':df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['interest_expense'],
#     'operating_cash_flow_to_debt':df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['total_liabilities'],
#     'cash_flow_margin_ratio_formula':df['operating_cash_flow']/df['total_liabilities'],
#     'cash_flow_to_debt':df['operating_cash_flow']/df['total_debt'],
#     'net_cash_flow_to_capital_expenditures':df['operating_cash_flow']/df['capex'],
#     'cash_flow_to_revenue':df['operating_cash_flow']/df['net_revenue'],
#     'cash_return_on_assets':df['operating_cash_flow']/df['total_assets'],
#     'cash_return_on_equity':df['operating_cash_flow']/df['total_se'],
#     'cash_to_income_ratio':df['operating_cash_flow']/df['operating_income'],
#     'cash_flow_per_share':df['operating_cash_flow']/df['shares_outstanding_non'],
#     'debt_payment':df['operating_cash_flow']/df['debt_repayment'],
#     'debt_coverage':df['operating_cash_flow']/df['long_term_debt'],
#     'cash_flow_from_operations_ratio':df['operating_cash_flow']/df['total_current_liabilities'],
#     'gross_profit_margin':df['net_revenue'] - df['cost_of_sales']/df['net_revenue'],
#     'receivables_turnover':df['net_revenue']/df['accounts_receivable'],
#     'capital_turnover_ratio':df['net_revenue']/df['working_capital_math'],
#     'assets_turnover_ratio':df['net_revenue']/df['total_assets'],
#     'accounts_receivableturnover':df['net_revenue']/df['accounts_receivable'],
#     'operating_cash_flow_to_debt':df['net_revenue']/df['total_se'],
#     'inventory_ratio':df['net_revenue']/df['inventory'],
#     'return_on_investment':df['net_income']+ df['interest_expense']/df['total_se'] + df['long_term_debt'],

#     'pretax_margin':df['net_income'] - df['income_tax_expense']/df['net_revenue'],
#     'income_to_net_worth_ratio':df['net_income'] - df['deferred_income_tax']/df['shares_outstanding_non'],
#     'return_on_assets_roa':df['net_income']/df['total_assets'],
#     'roe':df['net_income']/df['total_se'],
#     'profit_margin':df['net_income']/df['net_revenue'],
#     'earnings_per_share':df['net_income']/df['shares_outstanding_non'],
#     'current_cash_debt_coverage':df['net_cash_by_operating_activities']/df['total_current_liabilities'],
#     'cash_debt_coverage':df['net_cash_by_operating_activities']/df['total_liabilities'],
#     'long_term_debt_ratio':df['long_term_debt']/df['total_assets'],
#     'long_term_debt_equity_ratio':df['long_term_debt']/df['total_se'],
#     'lt_debt_as_pct_of_total_debt':df['long_term_debt']/df['total_liabilities'],
#     'inventory_pct_of_revenue':df['inventory']/df['net_revenue'],
#     'intangibles_pct_of_book_value':df['goodwill_n_intangible_assets']/df['total_se'],
#     'ffo_funds_from_operations_to_debt':df['ffo_math']/df['total_debt'],
#     'operating_margin':df['ebitda_non']-df['d_n_a']/df['net_revenue'],
#     'cash_coverage_ratio':df['ebitda_non']/df['interest_expense'],
#     'ebitda_per_share':df['ebitda_non']/df['shares_outstanding_non'],
#     'ebitda_interest_coverage':df['ebitda_non']/df['interest_expense'],
#     'ebitda_margin':df['ebitda_non']/df['net_revenue'],
#     'net_margin_profit_margin':df['ebitda_margin']/df['net_revenue'],
#     'return_on_capital_employed_ratio':df['ebit_math']/df['total_assets'] - df['total_current_liabilities'],
#     'debt_service_ratio':df['ebit_math']/df['interest_expense'],
#     'return_on_capital':df['ebit_math']/df['total_assets'],
#     'dividend_yield':df['dividends_paid']/df['shares_outstanding_non'],
#     'dividend_payout_ratio':df['dividends_paid']/df['net_income'],
#     'dividends_per_share':df['dividends_paid']/df['shares_outstanding_non'],
#     'inventory_turnover':df['cost_of_sales']/df['inventory'],
#     'cash_ratio':df['cash_non']/df['total_current_liabilities'],
#     'cash_flow_roa':df['operating_cash_flow']/df['total_assets'],
#     'average_collection_period':df['accounts_receivable']/df['net_revenue'],
#     'number_of_days_of_receivables':df['accounts_receivable']/df['inventory'],
#     'average_days_payables_outstanding':df['accounts_payable']/df['cost_of_sales'],
#     'days_sales_in_payables':df['accounts_payable']/df['total_opex'],    
# }

# url_to_name_map = {
#                     'net-working-capital-ratio': "Net Working Capital Ratio",
#                     'book-value-per-share': "Book value per share",
#                     'total-equity-to-total-assets': "Total equity to total_assets",
#                     'operating-cost-ratio': "Operating Cost Ratio",
#                     'perc​entage-of-debt-to-asset-formula': "Perc​entage of Debt to Asset Formula",

#                     'total-liabilities-pct-of-total-assets': "Total Liabilities % of total_assets",
#                     'debt-to-assets-ratio': "Debt-to-assets ratio",
#                     'debt-to-equity-ratio': "Debt-to-Equity Ratio",
#                     'quick-ratio': "Quick Ratio",
#                     'working-capital': "Working capital",
#                     'current-ratio': "Current Ratio",
#                     'capital-intensity': "Capital intensity",
#                     'equity-multiplier': "Equity Multiplier",
#                     'short-term-debt-to-equity-ratio': "Short Term Debt to Equity Ratio",
#                     'st-debt-as-pct-of-total-debt': "ST-Debt as % of Total Debt",
#                     'acid-test-ratio': "Acid Test Ratio",
#                     'pre-tax-income-to-sales': "Pre-tax Income to Sales",
#                     'pre-tax-return-on-assets': "Pre-tax return on assets",
#                     'pre-tax-return-on-common-equity': "Pre-tax return on common equity",
#                     'operating-roa': "Operating ROA",
#                     'operating-profit-margin': "Operating Profit margin",
#                     'free-operating-cash-flow-to-debt': "Free operating cash flow-to-debt",
#                     'discretionary-cash-flow-to-debt': "Discretionary cash flow-to-debt",
#                     'operating-cash-flow-to-interest': "operating_cash_flow to interest",
#                     'operating-cash-flow-to-debt': "operating_cash_flow to debt",
#                     'cash-flow-margin-ratio-formula': "Cash Flow Margin Ratio Formula",
#                     'cash-flow-to-debt': "Cash flow-to-debt",
#                     'net-cash-flow-to-capital-expenditures': "Net cash flow-to-capital expenditures",
#                     'cash-flow-to-revenue': "Cash flow to revenue",
#                     'cash-return-on-assets': "Cash return on assets",
#                     'cash-return-on-equity': "cash return on equity",
#                     'cash-to-income-ratio': "Cash to income ratio",
#                     'cash-flow-per-share': "Cash flow per share",
#                     'debt-payment': "Debt payment",
#                     'debt-coverage': "Debt Coverage",
#                     'cash-flow-from-operations-ratio': "Cash flow from operations ratio",
#                     'gross-profit-margin': "Gross profit margin",
#                     'rece​ivables-turnover': "Rece​ivables Turnover",
#                     'capital-turnover-ratio': "Capital Turnover Ratio",
#                     'assets-turnover-ratio': "Assets Turnover Ratio",
#                     'accounts-receivableturnover': "accounts_receivableturnover",
#                     'operating-cash-flow-to-debt': "operating_cash_flow to debt",
#                     'inventory-ratio': "Inventory Ratio",
#                     'return-on-investment': "Return on investment",

#                     'pretax-margin': "Pretax Margin",
#                     'income-to-net-worth-ratio': "Income to Net Worth Ratio",
#                     'return-on-assets-roa': "Return on Assets (ROA)",
#                     'roe': "ROE",
#                     'profit-margin': "Profit Margin",
#                     'earnings-per-share': "Earnings Per Share ",
#                     'current-cash-debt-coverage': "Current cash debt coverage",
#                     'cash-debt-coverage': "Cash debt coverage",
#                     'long-term-debt-ratio': "Long-Term Debt Ratio ",
#                     'long-term-debt-equity-ratio': "Long Term Debt/Equity Ratio",
#                     'lt-debt-as-pct-of-total-debt': "LT-Debt as % of Total Debt",
#                     'inventory-pct-of-revenue': "Inventory % of Revenue",
#                     'intangibles-pct-of-book-value': "Intangibles % of Book Value",
#                     'ffo-funds-from-operations-to-debt': "FFO (Funds from operations) to debt",
#                     'operating-margin': "Operating Margin",
#                     'cash-coverage-ratio': "Cash coverage ratio",
#                     'ebitda-per-share': "EBITDA per share",
#                     'ebitda-interest-coverage': "ebitda interest coverage",
#                     'ebitda-margin': "EBITDA Margin",
#                     'net-margin-profit-margin': "Net Margin (Profit Margin)",
#                     'return-on-capital-employed-ratio': "Return on Capital Employed Ratio",
#                     'debt-service-ratio': "Debt Service Ratio",
#                     'return-on-capital': "Return on capital",
#                     'dividend-yield': "Dividend Yield",
#                     'dividend-payout-ratio': "Dividend Payout Ratio",
#                     'dividends-per-share': "Dividends per share",
#                     'inventory-turnover': "inventory turnover",
#                     'cash-ratio': "Cash Ratio",
#                     'cash-flow-roa': "Cash Flow ROA",
#                     'average-collection-period': "Average Collection Period",
#                     'number-of-days-of-receivables': "Number of days of receivables",
#                     'average-days-payables-outstanding': "Average days payables outstanding",
#                     'days-sales-in-payables': "Days sales in payables",
# }

In [15]:
df['year'] = df['filing_date'].apply(lambda x: str(x)[0:4])
year_df = df.groupby("year").sum()

In [16]:
year_df = df.groupby("year").sum()

In [17]:
metric_name = url_to_metric_map['common-stock-repurchased']#['net-working-capital-ratio']
# metric_to_formula_map(df,metric_name)#.items()

KeyError: 'common-stock-repurchased'

In [32]:
url_to_metric_map['net-working-capital-ratio']

'net_working_capital_ratio'

In [33]:
# net_working_capital_ratio[url_to_metric_map['net-working-capital-ratio']]

In [34]:
matching_row

,index,subindex,Name,Financial Statement,URL,Variable Name,Title,Definition,Python Variable Name,Definitions Link,Also known as
38,39,6,cashAndCashEquivalents,Balance Sheet,cash-and-cash-equivalents,cash_and_cash_equivalents,Cash and Cash Equivalents,Cash equivalents are investments securities th...,cash_non,https://www.investopedia.com/terms/c/cashequiv...,NaN


In [21]:
# url_fin_metric = 'cash-and-cash-equivalents'
url_fin_metric = 'net-working-capital-ratio'
# fin_statements_matching = pd.read_csv("../reference_data/Financial_Statements_Reference_Matching.csv")#, encoding='cp1252')

In [24]:
# url_fin_metric = 'net-working-capital-ratio'

fin_metric_title = url_to_name_map[url_fin_metric]
fin_metric_name = url_fin_metric
metric_history = metric_to_formula_map(df,metric_name)
sorted_metric = metric_history.sort_values(ascending=True)

In [25]:
df['metric history'] = metric_history

In [29]:
df['metric history'] = metric_history
df['period'] = df['filing_date'].apply(lambda x: str(x)[0:4])
year_df = df.groupby("year").sum()

In [89]:
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Balance Sheet/quarter~JCP~balance_sheet.csv")
df = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Income Statement/quarter~JCP~income_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Cash Flow Statement/quarter~JCP~cash_flow_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Technology/quarter/Income Statement/quarter~AAPL~income_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Cash Flow Statement/quarter~JAX~cash_flow_statement.csv")
# jcp_bs

In [90]:
df

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,revenue,costOfRevenue,grossProfit,grossProfitRatio,...,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink
0,0,2020-08-01,JCP,2020-09-10 00:00:00,2020-09-10 17:17:52,Q2,-8244000000,-5281000000,-2963000000,0.359413,...,0.090490,-70000000.0,-676000000.0,0.081999,-2.08513,-2.08513,324200000,324200000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
1,0,2020-02-01,JCP,2020-03-20 00:00:00,2020-03-20 08:08:49,Q4,3493000000,2257000000,1236000000,0.353851,...,0.007157,-2000000.0,27000000.0,0.007730,0.08362,0.08362,322900000,322900000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
2,0,2019-11-02,JCP,2019-12-04 00:00:00,2019-12-04 16:30:46,Q3,2500000000,1541000000,959000000,0.383600,...,-0.037600,-1000000.0,-93000000.0,-0.037200,-0.28981,-0.28981,320900000,320900000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
3,0,2019-08-03,JCP,2019-08-28 00:00:00,2019-08-28 16:57:27,Q2,2619000000,1585000000,1034000000,0.394807,...,-0.016418,5000000.0,-48000000.0,-0.018328,-0.15028,-0.15028,319400000,319400000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
4,0,2019-05-04,JCP,2019-05-29 00:00:00,2019-05-29 16:31:20,Q1,2555000000,1630000000,925000000,0.362035,...,-0.059883,1000000.0,-154000000.0,-0.060274,-0.48473,-0.48473,317700000,317700000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0,2003-04-26,JCP,2003-06-10 00:00:00,2003-06-10 11:07:07,Q1,3711000000,2255000000,1456000000,0.392347,...,0.007545,8000000.0,61000000.0,0.016438,0.22369,0.22369,272700000,272700000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
70,0,2003-01-25,JCP,2003-04-10 00:00:00,2003-04-10 11:16:32,Q4,9549000000,6665000000,2884000000,0.302021,...,0.033302,116000000.0,202000000.0,0.021154,0.66667,0.66667,303000000,303000000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
71,0,2002-10-26,JCP,2002-12-06 00:00:00,2002-12-06 14:57:56,Q3,7872000000,5470000000,2402000000,0.305132,...,0.017785,51000000.0,123000000.0,0.015625,0.42123,0.42123,292000000,292000000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
72,0,2002-07-27,JCP,2002-09-06 00:00:00,2002-09-06 13:12:28,Q2,7198000000,5063000000,2135000000,0.296610,...,-0.001250,-3000000.0,-6000000.0,-0.000834,-0.02239,-0.02239,268000000,268000000,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...


In [92]:
if len(df) == len(df[df['revenue'] == 0]):
    return 404

18

In [88]:
filter_pos_neg = list(fin_statements_matching[fin_statements_matching['Name']=="revenue"]['positive_negative'])[0]
filter_pos_neg_list = ['positive','all', 'both', 'negative', 'zero_or_neg']

if "{}".format(filter_pos_neg) == "positive": #in filter_pos_neg_list:
    nan_rows = df[(df['revenue']<0)]
    nan_rows['revenue'] = np.nan
    df[(df['revenue']<0)] = nan_rows
#     df.loc[df['revenue'] < 0, 0:] = np.nan
#     df.loc[df['revenue'] < 0, 0:] = np.nan
#     df = df[~(df['revenue']<0)]
elif "{}".format(filter_pos_neg) == "neg": #in filter_pos_neg_list:
    nan_rows = df[(df['revenue']>=0)]
    nan_rows['revenue'] = np.nan
    df[(df['revenue']>=0)] = nan_rows
#     df.loc[df['revenue'] > 0, 0:] = np.nan
#     df = df[~(df['revenue']>=0)]
elif "{}".format(filter_pos_neg) == "zero_or_neg":
    nan_rows = df[(df['revenue']>0)]
    nan_rows['revenue'] = np.nan
    df[(df['revenue']>0)] = nan_rows
#     df.loc[df['revenue'] > 0, 0:] = np.nan
#     df = df[~(df['revenue']>0)]
elif "{}".format(filter_pos_neg) == "both":
    nan_rows = df[(df['revenue']==0)]
    nan_rows['revenue'] = np.nan
    df[(df['revenue']==0)] = nan_rows
#     df.loc[df['revenue'] == 0, 0:] = np.nan
#     df = df[~(df['revenue']==0)]
elif "{}".format(filter_pos_neg) == "all":
    pass
else:
    pass
df['revenue'] = df['revenue'].fillna(df['revenue'].rolling(4,center=True,min_periods=1).mean())

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,revenue,costOfRevenue,grossProfit,grossProfitRatio,...,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink
0,0.0,2020-08-01,JCP,2020-09-10 00:00:00,2020-09-10 17:17:52,Q2,NaN,-5.281000e+09,-2.963000e+09,0.359413,...,0.090490,-70000000.0,-676000000.0,0.081999,-2.08513,-2.08513,324200000.0,324200000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
1,0.0,2020-02-01,JCP,2020-03-20 00:00:00,2020-03-20 08:08:49,Q4,3.493000e+09,2.257000e+09,1.236000e+09,0.353851,...,0.007157,-2000000.0,27000000.0,0.007730,0.08362,0.08362,322900000.0,322900000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
2,0.0,2019-11-02,JCP,2019-12-04 00:00:00,2019-12-04 16:30:46,Q3,2.500000e+09,1.541000e+09,9.590000e+08,0.383600,...,-0.037600,-1000000.0,-93000000.0,-0.037200,-0.28981,-0.28981,320900000.0,320900000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
3,0.0,2019-08-03,JCP,2019-08-28 00:00:00,2019-08-28 16:57:27,Q2,2.619000e+09,1.585000e+09,1.034000e+09,0.394807,...,-0.016418,5000000.0,-48000000.0,-0.018328,-0.15028,-0.15028,319400000.0,319400000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
4,0.0,2019-05-04,JCP,2019-05-29 00:00:00,2019-05-29 16:31:20,Q1,2.555000e+09,1.630000e+09,9.250000e+08,0.362035,...,-0.059883,1000000.0,-154000000.0,-0.060274,-0.48473,-0.48473,317700000.0,317700000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.0,2003-04-26,JCP,2003-06-10 00:00:00,2003-06-10 11:07:07,Q1,3.711000e+09,2.255000e+09,1.456000e+09,0.392347,...,0.007545,8000000.0,61000000.0,0.016438,0.22369,0.22369,272700000.0,272700000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
70,0.0,2003-01-25,JCP,2003-04-10 00:00:00,2003-04-10 11:16:32,Q4,9.549000e+09,6.665000e+09,2.884000e+09,0.302021,...,0.033302,116000000.0,202000000.0,0.021154,0.66667,0.66667,303000000.0,303000000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
71,0.0,2002-10-26,JCP,2002-12-06 00:00:00,2002-12-06 14:57:56,Q3,7.872000e+09,5.470000e+09,2.402000e+09,0.305132,...,0.017785,51000000.0,123000000.0,0.015625,0.42123,0.42123,292000000.0,292000000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...
72,0.0,2002-07-27,JCP,2002-09-06 00:00:00,2002-09-06 13:12:28,Q2,7.198000e+09,5.063000e+09,2.135000e+09,0.296610,...,-0.001250,-3000000.0,-6000000.0,-0.000834,-0.02239,-0.02239,268000000.0,268000000.0,https://www.sec.gov/Archives/edgar/data/116612...,https://www.sec.gov/Archives/edgar/data/116612...


In [ ]:
df['revenue'] = df['revenue'].fillna(df['revenue'].rolling(4,center=True,min_periods=1).mean())

if df['revenue'][0]>=0 and (len(df['revenue'])) - df['revenue'].lt(0).sum() == 1:
    df.loc[df['revenue'] > 0, 0:] = np.nan
#     jcp_bs.replace()
elif df['revenue'][0]<=0 and (len(df['revenue'])) - df['revenue'].gt(0).sum()==1:
    df.loc[df['revenue']<0, 'revenue'] = np.nan
else:
    print("passed")

In [49]:
fin_statements_matching['positive_negative'].unique()

from collections import Counter

words = list(fin_statements_matching['positive_negative'])#[0]

print(Counter(words).keys())
print(Counter(words).values()) # counts the elements' frequency

dict_keys(['-', 'positive', 'all', 'both', 'negative', 'zero_or_neg', 'zero_or_pos', nan])
dict_values([17, 46, 15, 27, 2, 6, 1, 2])


In [18]:
# important
# jcp_bs['otherFinancingActivities'][::-1].values.searchsorted(0, side='right')
# list(jcp_bs['costAndExpenses'].pct_change())
# list(jcp_bs['costAndExpenses'])

In [50]:
early_missing_periods = jcp_bs[::-1]['commonStockRepurchased'].ne(0).idxmax()
jcp_bs = jcp_bs[0:early_missing_periods+1]

In [51]:
jcp_bs['commonStockRepurchased']

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
5           0.0
6           0.0
7           0.0
8           0.0
9           0.0
10          0.0
11          0.0
12          0.0
13          0.0
14          0.0
15     -50000.0
16          0.0
17   -3150000.0
Name: commonStockRepurchased, dtype: float64

In [53]:
# jcp_bs.loc[jcp_bs.commonStockRepurchased>0].iloc[[0,]].index[0]

In [52]:
# jcp_bs.loc[(jcp_bs.commonStockRepurchased>3).groupby(jcp_bs.symbol).idxmax]

In [48]:

ignore_periods = jcp_bs['commonStockRepurchased'][::-1].values.searchsorted(0, side='right')
df = jcp_bs[0:ignore_periods]

In [150]:
# jcp_bs['revenue']= 
# jcp_bs['revenue'].fillna(pd.rolling_mean(input_data_frame[var_list], 6, min_periods=1))
# jcp_bs['revenue'].fillna(pd.rolling(4,center=True,min_periods=1).mean((input_data_frame[var_list], 6, min_periods=1)))
# jcp_bs['revenue'].fillna(jcp_bs['revenue'].rolling(4, min_periods=1, center=True))
jcp_bs['revenue'] = jcp_bs['revenue'].fillna(jcp_bs['revenue'].rolling(4,center=True,min_periods=1).mean())
jcp_bs['revenue'] = jcp_bs['revenue'].replace(np.NaN, 0)

In [68]:
no_null_jcp = jcp_bs[~jcp_bs['retainedEarnings'].isnull()]
no_neg_jcp = no_null_jcp[~(no_null_jcp['retainedEarnings']<0)]
no_pos_jcp = no_neg_jcp[~(no_neg_jcp['retainedEarnings']>=0)]

In [116]:
# important
jcp_bs[(jcp_bs['otherFinancingActivites'].isnull())&(jcp_bs['otherFinancingActivites']<0)]
# jcp_bs[~(jcp_bs['retainedEarnings']<0)]

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,netIncome,depreciationAndAmortization,deferredIncomeTax,stockBasedCompensation,...,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink


In [79]:
if jcp_bs['revenue'][0]>=0 and (len(jcp_bs['revenue'])) - jcp_bs['revenue'].lt(0).sum() == 1:
    jcp_bs.loc[jcp_bs['revenue'] > 0, 0:] = np.nan
#     jcp_bs.replace()
elif jcp_bs['revenue'][0]<=0 and (len(jcp_bs['revenue'])) - jcp_bs['revenue'].gt(0).sum()==1:
    jcp_bs.loc[jcp_bs['revenue']<0, 'revenue'] = np.nan
else:
    print("passed")

In [155]:
jcp_bs[['totalOtherIncomeExpensesNet']].to_html().replace("<td>NaN</td>","<td></td>")

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>totalOtherIncomeExpensesNet</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td></td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>-4000000.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>14000000.0</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td>-8000000.0</td>\n    </tr>\n    <tr>\n      <th>10</th>\n      <td>2000000.0</td>\n    </tr>\n    <tr>\n      <th>11</th>\n      <td>14000000.0</td>\n    </tr>\n    <tr>\n      <th>12</th>\n      

In [8]:
# sorted_metric = np.sort(metric_history)[::-1]
fin_metric_name = 'cashAndCashEquivalents'
sorted_metric = df["{}".format(fin_metric_name)].sort_values()
quarters = round(len(sorted_metric)/4)
bottom_25 = sorted_metric[len(sorted_metric)-1-quarters]
top_25 = sorted_metric[quarters-1]
max_num = max(sorted_metric)
min_num = min(sorted_metric)
mean = sorted_metric.mean()
std_dev = sorted_metric.std()
std_dev_str = "+/-{}{}%".format(currency_symbol,round(abs((std_dev-mean)/mean)*100,1))
mean_str = magnitude_num(mean, currency_symbol)
max_str = magnitude_num(max_num, currency_symbol)
min_str = magnitude_num(min_num, currency_symbol)
bottom_25_str = magnitude_num(bottom_25, currency_symbol)
top_25_str = magnitude_num(top_25, currency_symbol)

In [9]:


latest_num = df["{}".format(fin_metric_name)][0]
first_num = list(df["{}".format(fin_metric_name)])[-1]
pct_chg = (latest_num - first_num)/first_num
historical_pct_chg = str(round(pct_chg*100, 1))

In [10]:
# latest_num = df["{}".format(fin_metric_name)][0]
# first_num = list(df["{}".format(fin_metric_name)])[-1]
# lifetime_sum_all_metric = df["{}".format(fin_metric_name)].sum()
# lifetime_sum_all_metric = magnitude_num(lifetime_sum_all_metric,currency_symbol)

In [11]:
# hide - cash equiv
if pct_chg>=0:
    pct_chg_str = "+{}%".format(pct_chg)
elif pct_chg<0:
    pct_chg_str = "-{}%".format(pct_chg)
else:
    pct_chg_str = ""
# historical_pct_chg = str(round(pct_chg*100, 1))
max_min_pct_diff = ((max_num-min_num)/min_num)
if max_min_pct_diff>=0:
    max_min_pct_diff_str = "+{}%".format(round(max_min_pct_diff)*100,1)
elif max_min_pct_diff<0:
    max_min_pct_diff_str = "-{}%".format(round(max_min_pct_diff)*100,1)
else:
    max_min_pct_diff_str = ""
    
df_fin_statement['Quarter & Year'] =(df_fin_statement['date'].astype(str).str[0:4]).astype(int)
latest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[0]
earliest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[-1]
earliest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[-1])
latest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[0])

In [13]:
fin_metric_title

'Cash and Cash Equivalents'

In [12]:
titles_bs =['Cash and Cash Equivalents',
 'Short Term Investments',
 'Cash And Short-Term Investments',
 'Accounts Receivables',
 'Inventory',
 'Other Current Assets',
 'Total Current Assets',
 'PP&E',
 'Goodwill',
 'Intangible Assets',
 'Goodwill and Intangible Assets',
 'Long-term Investments',
 'Tax Assets',
 'Other Non-Current Assets',
 'Total Non-Current Assets',
 'Other Assets',
 'Total Assets',
 'Accounts Payables',
 'Short-term Debt',
 'Income Tax Payables',
 'Deferred Revenue',
 'Other Current Liabilities',
 'Total Current Liabilities',
 'Long-term Debt',
 'Deferred Revenue Non-Current',
 'Deferred Tax Liabilities Non-Current',
 'Other Non-Current Liabilities',
 'Total Non-Current Liabilities',
 'Other Liabilities',
 'Total Liabilities',
 'Common Stock',
 'Retained Earnings',
 'Accumulated Other Comprehensive Income Loss',
 'Other Shareholders Equity',
 'Total Shareholders Equity',
 'Total Liabilities & Stockholders Equity',
 'Total Investments',
 'Total Debt',
 'Net Debt',
 'SEC Statement Link']
# col = titles_bs
# df_fin_statement = df_fin_statement[col]
# df_fin_statement = df_fin_statement.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]]
# cols

In [13]:
# hide - cash equiv
titles_list = ['Date','Symbol','Filing Date','Accepted Date','Period','SEC Filing Link'][:]

for x in reversed(titles_bs):
    if x in titles_list:
        titles_bs.remove(x)
        print(x, "is in")
    else:
        pass
#         print(x, "is not in")  
titles_bs.append('Quarter & Year') 
df_fin_statement = df_fin_statement.drop([ 'Unnamed: 0','date','symbol','fillingDate','acceptedDate','period','link'],axis=1, errors='ignore')
df_fin_statement.columns = titles_bs

# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_title = "Cash and Cash Equivalents"
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)

elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

In [15]:
# hide - variables
# vars_names_list = []
# vars_values_list = []
# # for name in vars().keys():
# #   vars_names_list.append(name)
# for name, value in globals().items():
#     vars_names_list.append(name)
#     vars_values_list.append(value)
# vars_names_list    
# vars_values_list_100 = [str(x)[0:50] for x in vars_values_list]
# vars_df = pd.DataFrame({'vars_names_list':vars_names_list,'vars_values_list_100':vars_values_list_100})
# # vars_values_list_100
# vars_df.to_csv("../reference_data/flask_variables.csv")

In [14]:
df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]
# pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
# df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
# # list(df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)])
# # len(df_pct_chg.transpose())
# # list(df_pct_chg['Quarter & Year'])
# df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

Quarter & Year,1994,1994,1994,1994,1994,1994,1994,1994,1994,1994,...,2019,2020,2020,2020,2020,2020,2020,2020,2020,2020
Cash and Cash Equivalents,-0.0438596,-0.05,0.0434783,-0.166667,-0.0438596,-0.05,0.0434783,-0.166667,-0.0438596,-0.05,...,-0.00995768,0.203415,-0.122041,NaN,0.203415,-0.122041,NaN,0.203415,-0.122041,NaN
Short Term Investments,-0.781609,0.600206,-0.876155,-0.281506,-0.781609,0.600206,-0.876155,-0.281506,-0.781609,0.600206,...,0.250831,-0.0966601,0.126873,NaN,-0.0966601,0.126873,NaN,-0.0966601,0.126873,NaN
Cash And Short-Term Investments,-0.0993485,-0.0237356,-0.2074,-0.200906,-0.0993485,-0.0237356,-0.2074,-0.200906,-0.0993485,-0.0237356,...,0.139403,0.0110293,0.0228935,NaN,0.0110293,0.0228935,NaN,0.0110293,0.0228935,NaN
Accounts Receivables,0.0516836,-0.192461,-0.01104,-0.0208206,0.0516836,-0.192461,-0.01104,-0.0208206,0.0516836,-0.192461,...,0.302148,-0.0435853,0.989764,NaN,-0.0435853,0.989764,NaN,-0.0435853,0.989764,NaN
Inventory,0.075188,0.0997451,0.00409041,0.101626,0.075188,0.0997451,0.00409041,0.101626,0.075188,0.0997451,...,0.228854,-0.16189,-0.0204383,NaN,-0.16189,-0.0204383,NaN,-0.16189,-0.0204383,NaN
Other Current Assets,0.229391,1.18167,-0.429092,1.15385,0.229391,1.18167,-0.429092,1.15385,0.229391,1.18167,...,-0.233573,0.428142,-0.662862,NaN,0.428142,-0.662862,NaN,0.428142,-0.662862,NaN
Total Current Assets,0.0380282,-0.0483535,-0.0511971,-0.0858361,0.0380282,-0.0483535,-0.0511971,-0.0858361,0.0380282,-0.0483535,...,0.135496,0.0263306,-0.0253839,NaN,0.0263306,-0.0253839,NaN,0.0263306,-0.0253839,NaN
PP&E,0.0180451,-0.00314795,0.00922844,-0.00301659,0.0180451,-0.00314795,0.00922844,-0.00301659,0.0180451,-0.00314795,...,0.00697949,0.00307861,0.192705,NaN,0.00307861,0.192705,NaN,0.00307861,0.192705,NaN
Goodwill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# hide
df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
# df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]
df_pct_chg = df_fin_statement

In [40]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
# df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
# df_pct_chg_t
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

# print("ww",len(col))
cols = list(df_pct_chg_t.columns)
print("ww",len(df))
cols = [cols[-1]] + cols[:-1]

df_pct_chg_t = df_pct_chg_t[cols]

df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index

df_t.index = range(len(df_t))
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
df_pct = df_pct_chg_t.to_html()#.replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

KeyError: 'Quarter & Year'

In [28]:

df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]

df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())

new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum#[1:] #take the data less the header row

df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
# df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
# df_t = df_t[0:25]


In [33]:

df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))

In [36]:
len(list(df_t))

429

In [39]:
df_t.columns

Index([1085.22,      '',    1994,    1994,    1994,    1994,    1994,    1994,
          1994,    1994,
       ...
          2019,    2020,    2020,    2020,    2020,    2020,    2020,    2020,
          2020,    2020],
      dtype='object', length=429)

In [38]:
df_t[df_t.columns]

,1085.22,,1994,1994,1994,1994,1994,1994,1994,1994,...,2019,2020,2020,2020,2020,2020,2020,2020,2020,2020
0,1085.22,Short Term Investments,19000000.00,87000000.00,54368000.00,439000000.00,19000000.00,87000000.00,54368000.00,439000000.00,...,67391000000.00,53877000000.00,59642000000.00,52927000000.00,53877000000.00,59642000000.00,52927000000.00,53877000000.00,59642000000.00,52927000000.00
1,1623530368000.00,Cash And Short-Term Investments,1106000000,1228000000,1257856000,1587000000,1106000000,1228000000,1257856000,1587000000,...,107162000000,94051000000,93025000000,90943000000,94051000000,93025000000,90943000000,94051000000,93025000000,90943000000
2,2748278856000.00,Accounts Receivables,1343000000,1277000000,1581347000,1599000000,1343000000,1277000000,1581347000,1599000000,...,39946000000,30677000000,32075000000,16120000000,30677000000,32075000000,16120000000,30677000000,32075000000,16120000000
3,1075288347000.00,Inventory,1287000000.00,1197000000.00,1088434000.00,1084000000.00,1287000000.00,1197000000.00,1088434000.00,1084000000.00,...,4097000000.00,3334000000.00,3978000000.00,4061000000.00,3334000000.00,3978000000.00,4061000000.00,3334000000.00,3978000000.00,4061000000.00
4,134638434000.00,Other Current Assets,686000000,558000000,255767000,448000000,686000000,558000000,255767000,448000000,...,12026000000,15691000000,10987000000,32589000000,15691000000,10987000000,32589000000,15691000000,10987000000,32589000000
5,547702767000.00,Total Current Assets,4422000000,4260000000,4476452000,4718000000,4422000000,4260000000,4476452000,4718000000,...,163231000000,143753000000,140065000000,143713000000,143753000000,140065000000,143713000000,143753000000,140065000000,143713000000
6,4517383452000.00,PP&E,677000000,665000000,667100000,661000000,677000000,665000000,667100000,661000000,...,44293000000,43986000000,43851000000,36766000000,43986000000,43851000000,36766000000,43986000000,43851000000,36766000000
7,1041800100000.00,Goodwill,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,97755000000.00,Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,103902000000.00,Goodwill and Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [19]:

# df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])


Nothing took 19.73895835876465 seconds


#### trying to read flask server with requests  

In [13]:
import requests
response = requests.get("http://www.google.com")
response = requests.get("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents")
print(response.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>NameError: name \'start_time\' is not defined // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script type="text/javascript">\n      var TRACEBACK = 1834011132928,\n          CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "t3wb8w77FxDSu3AUMLC3";\n    </

In [14]:
import urllib.request

myurl = "http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents"

# # GET is the default action
# response = urllib.request.urlopen(myurl)  

# # Output from the GET assuming response code was 200
# data = response.read()  

with urllib.request.urlopen("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents") as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    print(s)

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [36]:
import sys
print (sys.executable)
!{sys.executable} -m pip install requests_testadapter --upgrade

C:\Users\Tom\quantconnect\Scripts\python.exe
  Created wheel for requests-testadapter: filename=requests_testadapter-0.3.0-py3-none-any.whl size=4692 sha256=3b4e290717e164afb25bf11ba2539e3bd60e289e56538b60df70bcbe0399a55e
  Stored in directory: c:\users\tom\appdata\local\pip\cache\wheels\f6\85\0d\6b2d456f81dd6168b161711ebaca57d61fbe0744c52bcdb96b
Successfully built requests-testadapter


In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [15]:
import requests

os.environ['NO_PROXY'] = '127.0.0.1'
r = requests.get('http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents')
print(r.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'


In [40]:
import requests
os.environ['NO_PROXY'] = '127.0.1'

r=requests.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")

ConnectionError: HTTPConnectionPool(host='127.0.01', port=5000): Max retries exceeded with url: /aapl=stock/apple/cash-and-cash-equivalents (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020480EA1580>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [16]:
# BASE_URL='http://127.0.0.1:8000'
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents/"
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents"
resp=requests.get(BASE_URL)
# print(resp.status_code)
# print(resp.json())

In [17]:
resp.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [19]:
req = requests.post("http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents", 'html.parser')
req

<Response [404]>

In [48]:
from pypac import PACSession
session = PACSession()
# session.get('https://127.0.0.1/aapl=stock/apple/cash-and-cash-equival
session.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")
# <Response [200]>

ConnectionError: HTTPConnectionPool(host='127.0.01', port=5000): Max retries exceeded with url: /aapl=stock/apple/cash-and-cash-equivalents (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020480EFD550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
from pypac import PACSession
session = PACSession()
session.get('https://127.0.0/aapl=stock/apple/cash-and-cash-equivalents')
# <Response [200]>

In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [17]:








# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)


elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[titles_bs]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])

# return render_template('current_ratio.html', company_symbol = profiles_dict['symbol'],\
#                         company_long_name = profiles_dict['long name'],\
#                         company_currency = profiles_dict['currency'],\
#                         company_exchange = profiles_dict['exchange'],\
#                         company_industry = profiles_dict['industry'],\
#                         company_description = profiles_dict['description'],\
#                         company_sector = profiles_dict['sector'],\
#                         company_country = profiles_dict['country'],\
#                         company_ipo_date = profiles_dict['ipo date'],\
#                         company_short_name = profiles_dict['short name'],\
#                         first_num = first_num,\
#                         latest_num = latest_num,\
#                         historical_pct_chg = historical_pct_chg,\
#                         lifetime_sum_all_metric = lifetime_sum_all_metric,\
#                         mean_str = mean_str,\
#                         max_str = max_str,\
#                         min_str = min_str,\
#                         std_dev_str = std_dev_str,\
#                         bottom_25_str = bottom_25_str,\
#                         top_25_str = top_25_str,\
#                         earliest_year = earliest_year,\
#                         latest_year = latest_year,\
#                         earliest_metric = earliest_metric,\
#                         latest_metric = latest_metric,\
#                         max_min_pct_diff_str = max_min_pct_diff_str, df_bs_table_html = [df_table_html],df_html_tall = [df_html_tall],fin_metric_name = fin_metric_title,\
#                         df_date = df['date'].to_list(), df_rev = df["{}".format(fin_metric_name)].to_list(),\
#                         df_json  =df.to_numpy().tolist(),\
#                         table_pct = [df_pct],\
#                         tables=[df_html],\
#                         titles=df.columns.values,\
#                         total_time=total_seconds,\
#                         place_name=url_symbol,\
#                         max=17000,\
#                         labels=labels,\
#                         values=values)


KeyError: "None of [Index(['Cash and Cash Equivalents', 'Short Term Investments',\n       'Cash And Short-Term Investments', 'Accounts Receivables', 'Inventory',\n       'Other Current Assets', 'Total Current Assets', 'PP&E', 'Goodwill',\n       'Intangible Assets', 'Goodwill and Intangible Assets',\n       'Long-term Investments', 'Tax Assets', 'Other Non-Current Assets',\n       'Total Non-Current Assets', 'Other Assets', 'Total Assets',\n       'Accounts Payables', 'Short-term Debt', 'Income Tax Payables',\n       'Deferred Revenue', 'Other Current Liabilities',\n       'Total Current Liabilities', 'Long-term Debt',\n       'Deferred Revenue Non-Current', 'Deferred Tax Liabilities Non-Current',\n       'Other Non-Current Liabilities', 'Total Non-Current Liabilities',\n       'Other Liabilities', 'Total Liabilities', 'Common Stock',\n       'Retained Earnings', 'Accumulated Other Comprehensive Income Loss',\n       'Other Shareholders Equity', 'Total Shareholders Equity',\n       'To

In [18]:
cols

['',
 -0.0005012531328321135,
 -0.0005010020040080443,
 -0.0005007511266900133,
 -0.0005005005005005447,
 -0.0005002501250624958,
 -0.0004999999999999449,
 -0.0004997501249375258,
 -0.0004995004995005381,
 -0.0004992511233150587,
 -0.0004990019960079417,
 -0.0004987531172069293,
 -0.0004985044865404298,
 -0.0004982561036372957,
 -0.0004980079681274896,
 -0.0004977600796416404,
 -0.0004975124378109319,
 -0.0004972650422675473,
 -0.0004970178926441138,
 -0.0004967709885742577,
 -0.0004965243296921606,
 -0.0004962779156327812,
 -0.0004960317460317443,
 -0.0004957858205255627,
 -0.0004955401387511937,
 -0.0004952947003467045,
 -0.0004950495049504955,
 nan]

In [1]:








# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)


elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[titles_bs]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])

# return render_template('current_ratio.html', company_symbol = profiles_dict['symbol'],\
#                         company_long_name = profiles_dict['long name'],\
#                         company_currency = profiles_dict['currency'],\
#                         company_exchange = profiles_dict['exchange'],\
#                         company_industry = profiles_dict['industry'],\
#                         company_description = profiles_dict['description'],\
#                         company_sector = profiles_dict['sector'],\
#                         company_country = profiles_dict['country'],\
#                         company_ipo_date = profiles_dict['ipo date'],\
#                         company_short_name = profiles_dict['short name'],\
#                         first_num = first_num,\
#                         latest_num = latest_num,\
#                         historical_pct_chg = historical_pct_chg,\
#                         lifetime_sum_all_metric = lifetime_sum_all_metric,\
#                         mean_str = mean_str,\
#                         max_str = max_str,\
#                         min_str = min_str,\
#                         std_dev_str = std_dev_str,\
#                         bottom_25_str = bottom_25_str,\
#                         top_25_str = top_25_str,\
#                         earliest_year = earliest_year,\
#                         latest_year = latest_year,\
#                         earliest_metric = earliest_metric,\
#                         latest_metric = latest_metric,\
#                         max_min_pct_diff_str = max_min_pct_diff_str, df_bs_table_html = [df_table_html],df_html_tall = [df_html_tall],fin_metric_name = fin_metric_title,\
#                         df_date = df['date'].to_list(), df_rev = df["{}".format(fin_metric_name)].to_list(),\
#                         df_json  =df.to_numpy().tolist(),\
#                         table_pct = [df_pct],\
#                         tables=[df_html],\
#                         titles=df.columns.values,\
#                         total_time=total_seconds,\
#                         place_name=url_symbol,\
#                         max=17000,\
#                         labels=labels,\
#                         values=values)


NameError: name 'df_fin_statement' is not defined

In [22]:
cols

['',
 0.0,
 0.0,
 0.0,
 -0.0005012531328321135,
 0.0,
 0.0,
 0.0,
 -0.0005012531328321135,
 0.0,
 0.0,
 0.0,
 -0.0005012531328321135,
 0.0,
 0.0,
 0.0,
 -0.0005012531328321135,
 0.0,
 0.0,
 0.0,
 -0.0005010020040080443,
 0.0,
 0.0,
 0.0,
 -0.0005010020040080443,
 0.0,
 0.0,
 0.0,
 -0.0005010020040080443,
 0.0,
 0.0,
 0.0,
 -0.0005010020040080443,
 0.0,
 0.0,
 0.0,
 -0.0005007511266900133,
 0.0,
 0.0,
 0.0,
 -0.0005007511266900133,
 0.0,
 0.0,
 0.0,
 -0.0005007511266900133,
 0.0,
 0.0,
 0.0,
 -0.0005007511266900133,
 0.0,
 0.0,
 0.0,
 -0.0005005005005005447,
 0.0,
 0.0,
 0.0,
 -0.0005005005005005447,
 0.0,
 0.0,
 0.0,
 -0.0005005005005005447,
 0.0,
 0.0,
 0.0,
 -0.0005005005005005447,
 0.0,
 0.0,
 0.0,
 -0.0005002501250624958,
 0.0,
 0.0,
 0.0,
 -0.0005002501250624958,
 0.0,
 0.0,
 0.0,
 -0.0005002501250624958,
 0.0,
 0.0,
 0.0,
 -0.0005002501250624958,
 0.0,
 0.0,
 -0.0004999999999999449,
 0.0,
 0.0,
 -0.0004999999999999449,
 0.0,
 0.0,
 -0.0004999999999999449,
 0.0,
 0.0,
 0.0,
 0.0,


In [19]:
len(cols)

28